In [ ]:
# !pip install xgboost

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, classification_report

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Load datasets
file_path1 = "/content/drive/MyDrive/Hackathon_2025/employee_train.csv"
train_df = pd.read_csv(file_path1, header=0, index_col=None)

file_path2 = "/content/drive/MyDrive/Hackathon_2025/employee_test.csv"
test_df = pd.read_csv(file_path2, header=0, index_col=None)

In [4]:
train_df.shape

(1048, 28)

In [5]:
test_df.shape

(210, 27)

In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048 entries, 0 to 1047
Data columns (total 28 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   S.No                       1048 non-null   int64 
 1   Timestamp                  1048 non-null   object
 2   Age                        1048 non-null   int64 
 3   Gender                     1048 non-null   object
 4   Country                    1048 non-null   object
 5   state                      636 non-null    object
 6   self_employed              1030 non-null   object
 7   family_history             1048 non-null   object
 8   treatment                  1048 non-null   object
 9   work_interfere             812 non-null    object
 10  no_employees               1048 non-null   object
 11  remote_work                1048 non-null   object
 12  tech_company               1048 non-null   object
 13  benefits                   1035 non-null   object
 14  care_opt

In [7]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210 entries, 0 to 209
Data columns (total 27 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   S.No                       210 non-null    int64 
 1   Timestamp                  210 non-null    object
 2   Age                        210 non-null    int64 
 3   Gender                     210 non-null    object
 4   Country                    210 non-null    object
 5   state                      107 non-null    object
 6   self_employed              210 non-null    object
 7   family_history             210 non-null    object
 8   work_interfere             182 non-null    object
 9   no_employees               210 non-null    object
 10  remote_work                210 non-null    object
 11  tech_company               210 non-null    object
 12  benefits                   207 non-null    object
 13  care_options               210 non-null    object
 14  wellness_p

In [8]:
for i in train_df.columns:
    print(i, "--"*25)
    print(train_df[i].value_counts())
    print()

S.No --------------------------------------------------
S.No
1048    1
1       1
2       1
3       1
4       1
       ..
28      1
29      1
30      1
31      1
32      1
Name: count, Length: 1048, dtype: int64

Timestamp --------------------------------------------------
Timestamp
8/27/2014 12:31    8
8/27/2014 15:23    8
8/27/2014 15:24    8
8/27/2014 11:43    7
8/27/2014 12:34    6
                  ..
8/29/2014 9:59     1
8/29/2014 10:03    1
8/27/2014 12:03    1
8/27/2014 12:07    1
8/27/2014 12:10    1
Name: count, Length: 679, dtype: int64

Age --------------------------------------------------
Age
 29             73
 32             64
 33             63
 26             63
 27             61
 31             57
 28             56
 30             56
 34             52
 35             49
 25             48
 23             41
 24             38
 37             36
 36             31
 38             30
 40             26
 39             26
 43             23
 22             20
 42    

In [9]:
pd.set_option('display.max_columns', None)
train_df.head(10)

,S.No,Timestamp,Age,Gender,Country,state,self_employed,family_history,treatment,work_interfere,no_employees,remote_work,tech_company,benefits,care_options,wellness_program,seek_help,anonymity,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence,comments
0,1,8/27/2014 11:29,37,Female,United States,IL,NaN,No,Yes,Often,25-Jun,No,Yes,Yes,Not sure,No,Yes,Yes,Somewhat easy,No,No,Some of them,Yes,No,Maybe,Yes,No,NaN
1,2,8/27/2014 11:29,44,M,United States,IN,NaN,No,No,Rarely,More than 1000,No,No,Don't know,No,Don't know,Don't know,Don't know,Don't know,Maybe,No,No,No,No,No,Don't know,No,NaN
2,3,8/27/2014 11:29,32,Male,Canada,NaN,NaN,No,No,Rarely,25-Jun,No,Yes,No,No,No,No,Don't know,Somewhat difficult,No,No,Yes,Yes,Yes,Yes,No,No,NaN
3,4,8/27/2014 11:29,31,Male,United Kingdom,NaN,NaN,Yes,Yes,Often,26-100,No,Yes,No,Yes,No,No,No,Somewhat difficult,Yes,Yes,Some of them,No,Maybe,Maybe,No,Yes,NaN
4,5,8/27/2014 11:30,31,Male,United States,TX,NaN,No,No,Never,100-500,Yes,Yes,Yes,No,Don't know,Don't know,Don't know,Don't know,No,No,Some of them,Yes,Yes,Yes,Don't know,No,NaN
5,6,8/27/2014 11:31,33,Male,United States,TN,NaN,Yes,No,Sometimes,25-Jun,No,Yes,Yes,Not sure,No,Don't know,Don't know,Don't know,No,No,Yes,Yes,No,Maybe,Don't know,No,NaN
6,7,8/27/2014 11:31,35,Female,United States,MI,NaN,Yes,Yes,Sometimes,5-Jan,Yes,Yes,No,No,No,No,No,Somewhat difficult,Maybe,Maybe,Some of them,No,No,No,Don't know,No,NaN
7,8,8/27/2014 11:32,39,M,Canada,NaN,NaN,No,No,Never,5-Jan,Yes,Yes,No,Yes,No,No,Yes,Don't know,No,No,No,No,No,No,No,No,NaN
8,9,8/27/2014 11:32,42,Female,United States,IL,NaN,Yes,Yes,Sometimes,100-500,No,Yes,Yes,Yes,No,No,No,Very difficult,Maybe,No,Yes,Yes,No,Maybe,No,No,NaN
9,10,8/27/2014 11:32,23,Male,Canada,NaN,NaN,No,No,Never,26-100,No,Yes,Don't know,No,Don't know,Don't know,Don't know,Don't know,No,No,Yes,Yes,Maybe,Maybe,Yes,No,NaN


### Replacing Invalid values with mean in Age Variable

In [10]:
# Calculate mean age (only from valid values)
mean_age = round(train_df.loc[(train_df['Age'] >= 0) & (train_df['Age'] <= 120), 'Age'].mean())

# Replace invalid ages with rounded mean
train_df['Age'] = train_df['Age'].apply(lambda x: mean_age if (x < 0 or x > 120) else x)

# Verify Age Column
print("🔹 Unique Age Values:", train_df['Age'].unique())

🔹 Unique Age Values: [37 44 32 31 33 35 39 42 23 29 36 27 46 41 34 30 40 38 50 24 18 28 26 22
 19 25 45 21 43 56 60 54 55 48 20 57 58 47 62 51 65 49  5 53 61  8]


In [11]:
for i in test_df.columns:
    print(i, "--"*25)
    print(test_df[i].value_counts())
    print()

S.No --------------------------------------------------
S.No
1      1
2      1
3      1
4      1
5      1
      ..
206    1
207    1
208    1
209    1
210    1
Name: count, Length: 210, dtype: int64

Timestamp --------------------------------------------------
Timestamp
8/29/2014 11:32     2
8/29/2014 17:04     2
8/29/2014 20:53     2
2/21/2015 4:16      2
2/21/2015 5:55      2
                   ..
9/12/2015 11:17     1
9/26/2015 1:07      1
11/7/2015 12:36     1
11/30/2015 21:25    1
2/1/2016 23:04      1
Name: count, Length: 204, dtype: int64

Age --------------------------------------------------
Age
 32    18
 34    13
 25    13
 29    12
 28    12
 26    12
 23    10
 31    10
 27    10
 38     9
 24     8
 39     7
 37     7
 40     7
 33     7
 30     7
 35     6
 36     6
 41     5
 46     5
 43     5
 42     3
 44     3
 45     3
 20     2
 48     1
 11     1
 50     1
-1      1
 21     1
 51     1
 72     1
 56     1
 60     1
 22     1
Name: count, dtype: int64

Gender ----

### Combining different gender values in to 3 class

#### Define a Function to Normalize Gender

In [12]:
def classify_gender(value):
    """Classifies gender into 'Male', 'Female', or 'Other'."""
    value = str(value).strip().lower()  # Convert to lowercase & remove spaces

    # Male Categories
    male_terms = ["male", "m", "man", "guy", "mal", "mail", "msle", "maile", "malr", "male-ish", "cis male", "male (cis)"]

    # Female Categories
    female_terms = ["female", "f", "woman", "femake", "cis female", "female (cis)", "female (trans)", "trans woman"]

    # Classification
    if value in male_terms:
        return "Male"
    elif value in female_terms:
        return "Female"
    else:
        return "Other"

#### Apply Function to train_df and test_df

In [13]:
# Apply to train dataset
train_df["Gender"] = train_df["Gender"].apply(classify_gender)

# Apply to test dataset
test_df["Gender"] = test_df["Gender"].apply(classify_gender)

# Check unique values after transformation
print("🔹 Train Dataset Gender Counts:\n", train_df["Gender"].value_counts())
print("\n🔹 Test Dataset Gender Counts:\n", test_df["Gender"].value_counts())

🔹 Train Dataset Gender Counts:
 Gender
Male      823
Female    204
Other      21
Name: count, dtype: int64

🔹 Test Dataset Gender Counts:
 Gender
Male      162
Female     44
Other       4
Name: count, dtype: int64


## Creating a Function to Standardize the Values of No. of employees column

In [14]:
def standardize_no_employees(value):
    """Converts inconsistent employee range formats into standardized ranges."""
    value = str(value).strip().lower()  # Convert to lowercase & remove spaces

    # Define mappings
    if value in ["5-jan", "jan-5", "1-5"]:
        return "1-5"
    elif value in ["25-jun", "jun-25", "6-25"]:
        return "6-25"
    elif value == "26-100":
        return "26-100"
    elif value == "100-500":
        return "100-500"
    elif value == "500-1000":
        return "500-1000"
    elif value == "more than 1000":
        return "1000+"
    else:
        return "Unknown"  # Handle unexpected values

#### Apply Function to train_df and test_df

In [15]:
# Apply function to train & test datasets
train_df["no_employees"] = train_df["no_employees"].apply(standardize_no_employees)
test_df["no_employees"] = test_df["no_employees"].apply(standardize_no_employees)

# Check unique values after transformation
print("🔹 Train Dataset Employee Count Categories:\n", train_df["no_employees"].value_counts())
print("\n🔹 Test Dataset Employee Count Categories:\n", test_df["no_employees"].value_counts())

🔹 Train Dataset Employee Count Categories:
 no_employees
6-25        245
26-100      238
1000+       227
100-500     150
1-5         133
500-1000     55
Name: count, dtype: int64

🔹 Test Dataset Employee Count Categories:
 no_employees
1000+       55
26-100      51
6-25        45
1-5         28
100-500     26
500-1000     5
Name: count, dtype: int64


### Filling the missing values in the dataframe for both Train and Test Datasets

In [16]:
train_df.isnull().sum()

,0
S.No,0
Timestamp,0
Age,0
Gender,0
Country,0
state,412
self_employed,18
family_history,0
treatment,0
work_interfere,236


In [17]:
# For Train dataset

train_df.fillna({"state": train_df["state"].mode()[0]}, inplace=True)
train_df.fillna({"self_employed": train_df["self_employed"].mode()[0]}, inplace=True)
train_df.fillna({"work_interfere": train_df["work_interfere"].mode()[0]}, inplace=True)
train_df.fillna({"benefits": train_df["benefits"].mode()[0]}, inplace=True)
train_df.fillna({"wellness_program": train_df["wellness_program"].mode()[0]}, inplace=True)
train_df.fillna({"leave": train_df["leave"].mode()[0]}, inplace=True)

In [18]:
test_df.isnull().sum()

,0
S.No,0
Timestamp,0
Age,0
Gender,0
Country,0
state,103
self_employed,0
family_history,0
work_interfere,28
no_employees,0


In [19]:
# For test dataset

test_df.fillna({"state": test_df["state"].mode()[0]}, inplace=True)
test_df.fillna({"work_interfere": test_df["work_interfere"].mode()[0]}, inplace=True)
test_df.fillna({"benefits": test_df["benefits"].mode()[0]}, inplace=True)
test_df.fillna({"wellness_program": test_df["wellness_program"].mode()[0]}, inplace=True)
test_df.fillna({"leave": test_df["leave"].mode()[0]}, inplace=True)

### Drop irrelevant columns

In [20]:
train_df.drop(columns=['S.No', 'Timestamp','Country', 'state','comments'], inplace=True, errors='ignore')
test_df.drop(columns=['S.No', 'Timestamp','Country', 'state','comments'], inplace=True, errors='ignore')

In [21]:
train_df.shape

(1048, 23)

In [22]:
test_df.shape

(210, 22)

#### Label Encoding

In [23]:
colname=[]
for x in train_df.columns:
    if train_df[x].dtype=='object':
        colname.append(x)

print(colname)

['Gender', 'self_employed', 'family_history', 'treatment', 'work_interfere', 'no_employees', 'remote_work', 'tech_company', 'benefits', 'care_options', 'wellness_program', 'seek_help', 'anonymity', 'leave', 'mental_health_consequence', 'phys_health_consequence', 'coworkers', 'supervisor', 'mental_health_interview', 'phys_health_interview', 'mental_vs_physical', 'obs_consequence']


In [24]:
le=preprocessing.LabelEncoder()

for x in colname:
    train_df[x]=le.fit_transform(train_df[x])

In [25]:
train_df.columns

Index(['Age', 'Gender', 'self_employed', 'family_history', 'treatment',
       'work_interfere', 'no_employees', 'remote_work', 'tech_company',
       'benefits', 'care_options', 'wellness_program', 'seek_help',
       'anonymity', 'leave', 'mental_health_consequence',
       'phys_health_consequence', 'coworkers', 'supervisor',
       'mental_health_interview', 'phys_health_interview',
       'mental_vs_physical', 'obs_consequence'],
      dtype='object')

### Separating X and Y variables

In [26]:
test_df.columns

Index(['Age', 'Gender', 'self_employed', 'family_history', 'work_interfere',
       'no_employees', 'remote_work', 'tech_company', 'benefits',
       'care_options', 'wellness_program', 'seek_help', 'anonymity', 'leave',
       'mental_health_consequence', 'phys_health_consequence', 'coworkers',
       'supervisor', 'mental_health_interview', 'phys_health_interview',
       'mental_vs_physical', 'obs_consequence'],
      dtype='object')

In [27]:
colname=[]
for x in test_df.columns:
    if test_df[x].dtype=='object':
        colname.append(x)

print(colname)

['Gender', 'self_employed', 'family_history', 'work_interfere', 'no_employees', 'remote_work', 'tech_company', 'benefits', 'care_options', 'wellness_program', 'seek_help', 'anonymity', 'leave', 'mental_health_consequence', 'phys_health_consequence', 'coworkers', 'supervisor', 'mental_health_interview', 'phys_health_interview', 'mental_vs_physical', 'obs_consequence']


In [28]:
for x in colname:
  test_df[x]=le.fit_transform(test_df[x])

X_test = test_df[['Age', 'Gender', 'self_employed',
       'family_history', 'work_interfere', 'no_employees', 'remote_work',
       'tech_company', 'benefits', 'care_options', 'wellness_program',
       'seek_help', 'anonymity', 'leave', 'mental_health_consequence',
       'phys_health_consequence', 'coworkers', 'supervisor',
       'mental_health_interview', 'phys_health_interview',
       'mental_vs_physical', 'obs_consequence']]

In [29]:
df = pd.DataFrame(train_df)

# Reorder columns
desired_order = ['Age', 'Gender', 'self_employed',
       'family_history', 'work_interfere', 'no_employees',
       'remote_work', 'tech_company', 'benefits', 'care_options',
       'wellness_program', 'seek_help', 'anonymity', 'leave',
       'mental_health_consequence', 'phys_health_consequence', 'coworkers',
       'supervisor', 'mental_health_interview', 'phys_health_interview',
       'mental_vs_physical', 'obs_consequence', 'treatment']  # Defining the desired order of columns
df_reordered_train = df[desired_order]

# Display the reordered DataFrame
df_reordered_train.head()

,Age,Gender,self_employed,family_history,work_interfere,no_employees,remote_work,tech_company,benefits,care_options,wellness_program,seek_help,anonymity,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence,treatment
0,37,0,0,0,1,5,0,1,2,1,1,2,2,2,1,1,1,2,1,0,2,0,1
1,44,1,0,0,2,2,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0
2,32,1,0,0,2,5,0,1,1,0,1,1,0,1,1,1,2,2,2,2,1,0,0
3,31,1,0,1,1,3,0,1,1,2,1,1,1,1,2,2,1,0,0,0,1,1,1
4,31,1,0,0,0,1,1,1,2,0,0,0,0,0,1,1,1,2,2,2,0,0,0


In [30]:
# Creating X and Y variables

X = df_reordered_train.values[:,0:-1]
Y = df_reordered_train.values[:,-1]
Y = Y.astype(int)

In [31]:
print(X.shape)
print(Y.shape)

(1048, 22)
(1048,)


In [32]:
# Scaling the Data

scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

### Splitting the data into train and test

In [33]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=10)

In [34]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(838, 22)
(210, 22)
(838,)
(210,)


### Training the random Forest Model

In [35]:
rf = RandomForestClassifier(n_estimators=300, random_state=42)
rf.fit(X_train, Y_train)

RandomForestClassifier(n_estimators=300, random_state=42)

In [36]:
# Prediction on test data

y_pred = rf.predict(X_test)

In [37]:
# Evaluating the Model

accuracy = accuracy_score(Y_test, y_pred)

f1 = f1_score(Y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}, F1 Score: {f1:.4f}")

print(classification_report(Y_test, y_pred))

Accuracy: 0.7619, F1 Score: 0.7619
              precision    recall  f1-score   support

           0       0.78      0.75      0.76       107
           1       0.75      0.78      0.76       103

    accuracy                           0.76       210
   macro avg       0.76      0.76      0.76       210
weighted avg       0.76      0.76      0.76       210



### Tuning the RF model using GridsearchCV

In [38]:
# Define parameter grid for hyperparameter tuning
param_grid = {
    'n_estimators': [100, 200, 300, 400, 500],  # Number of trees in forest
    'max_depth': [5, 7, 10, 14, 18],   # Control overfitting
    'min_samples_split': [2, 5, 7, 10, 12],  # Minimum samples required to split a node
    'min_samples_leaf': [1, 2, 3, 4, 5],    # Minimum samples at a leaf node
    'max_features': ['sqrt', 'log2'], # Number of features per split
}

In [39]:
# Initialize Random Forest
rf = RandomForestClassifier(random_state=42)

# Using Randomized Search for Hyperparameter Tuning
rf_search = RandomizedSearchCV(
    rf, param_grid,
    cv=5, n_iter=15, scoring='accuracy',
    verbose=2, random_state=42, n_jobs=-1)

In [40]:
rf_search.fit(X_train, Y_train)

Fitting 5 folds for each of 15 candidates, totalling 75 fits


RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42),
                   n_iter=15, n_jobs=-1,
                   param_distributions={'max_depth': [5, 7, 10, 14, 18],
                                        'max_features': ['sqrt', 'log2'],
                                        'min_samples_leaf': [1, 2, 3, 4, 5],
                                        'min_samples_split': [2, 5, 7, 10, 12],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500]},
                   random_state=42, scoring='accuracy', verbose=2)

In [41]:
# Get best model
best_rf = rf_search.best_estimator_

In [42]:
# Train the optimized model
best_rf.fit(X_train, Y_train)

RandomForestClassifier(max_depth=5, min_samples_leaf=2, min_samples_split=10,
                       n_estimators=400, random_state=42)

In [43]:
# Evaluate performance
y_pred = best_rf.predict(X_test)

In [44]:
from sklearn.metrics import accuracy_score, classification_report
print("Optimized Random Forest Accuracy:", accuracy_score(Y_test, y_pred))
print(classification_report(Y_test, y_pred))

Optimized Random Forest Accuracy: 0.7476190476190476
              precision    recall  f1-score   support

           0       0.76      0.74      0.75       107
           1       0.74      0.76      0.75       103

    accuracy                           0.75       210
   macro avg       0.75      0.75      0.75       210
weighted avg       0.75      0.75      0.75       210



## Predictions on Test Data

In [45]:
test_df.head()

,Age,Gender,self_employed,family_history,work_interfere,no_employees,remote_work,tech_company,benefits,care_options,wellness_program,seek_help,anonymity,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence
0,39,1,1,1,3,0,1,1,1,2,2,2,2,1,1,1,2,2,1,0,2,1
1,26,0,0,1,3,2,0,1,2,2,1,1,0,0,1,1,1,2,1,0,1,1
2,23,0,0,1,3,3,0,0,1,0,1,1,0,1,2,1,0,1,1,0,1,0
3,35,1,0,1,1,2,0,1,0,1,1,1,2,4,1,1,1,1,1,0,1,0
4,36,1,0,0,0,0,1,1,0,1,0,0,0,4,1,1,1,1,1,1,0,0


In [46]:
# Scaling the data

test = test_df.values
test = scaler.transform(test)

In [47]:
# Training the model on train Dataset
rf = RandomForestClassifier(n_estimators=300, random_state=42)

# fit the model on the data and predict the values

rf.fit(X_train, Y_train)

RandomForestClassifier(n_estimators=300, random_state=42)

In [48]:
# Making Final predictions on Test dataset

test_pred = rf.predict(test)
test_pred

array([1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1,
       1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0,
       1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1,
       1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1,
       1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0,
       0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1])

In [49]:
# Convert 1 -> 'Yes', 0 -> 'No'
test_pred_labels = np.where(test_pred == 1, "Yes", "No")
print(test_pred_labels)

['Yes' 'Yes' 'Yes' 'Yes' 'No' 'No' 'Yes' 'No' 'No' 'Yes' 'Yes' 'No' 'No'
 'No' 'No' 'Yes' 'Yes' 'No' 'No' 'No' 'No' 'No' 'No' 'Yes' 'No' 'Yes'
 'Yes' 'Yes' 'Yes' 'Yes' 'No' 'Yes' 'Yes' 'Yes' 'Yes' 'Yes' 'No' 'No' 'No'
 'No' 'Yes' 'No' 'No' 'Yes' 'Yes' 'No' 'Yes' 'Yes' 'Yes' 'No' 'No' 'No'
 'Yes' 'Yes' 'No' 'No' 'No' 'No' 'Yes' 'Yes' 'No' 'Yes' 'No' 'No' 'Yes'
 'No' 'Yes' 'No' 'Yes' 'Yes' 'No' 'Yes' 'No' 'No' 'Yes' 'No' 'Yes' 'No'
 'Yes' 'No' 'No' 'Yes' 'No' 'Yes' 'Yes' 'Yes' 'No' 'Yes' 'Yes' 'No' 'No'
 'Yes' 'No' 'No' 'Yes' 'No' 'Yes' 'Yes' 'Yes' 'Yes' 'No' 'Yes' 'No' 'Yes'
 'Yes' 'Yes' 'No' 'No' 'No' 'Yes' 'No' 'Yes' 'No' 'No' 'Yes' 'Yes' 'Yes'
 'Yes' 'No' 'No' 'Yes' 'No' 'No' 'Yes' 'No' 'No' 'Yes' 'Yes' 'No' 'No'
 'No' 'Yes' 'Yes' 'Yes' 'Yes' 'Yes' 'No' 'No' 'Yes' 'No' 'Yes' 'Yes' 'Yes'
 'No' 'No' 'No' 'Yes' 'Yes' 'Yes' 'Yes' 'No' 'Yes' 'Yes' 'No' 'No' 'No'
 'No' 'Yes' 'No' 'Yes' 'Yes' 'Yes' 'No' 'Yes' 'No' 'Yes' 'No' 'No' 'No'
 'No' 'No' 'Yes' 'Yes' 'Yes' 'No' 'Yes' 'Yes' 'No' 'Yes'

In [50]:
predictions_df = pd.DataFrame({'S.No': range(1, 211),
                               'treatment': test_pred_labels})

In [51]:
file_path = "/content/drive/MyDrive/Hackathon_2025/RF_Pred.csv"

In [52]:
predictions_df.to_csv(file_path, index=False)
print(f"File saved at: {file_path}")

File saved at: /content/drive/MyDrive/Hackathon_2025/RF_Pred.csv
